In [1]:
import os
import csv
import torch
import random
import timeit
import torchaudio
import numpy as np
import torch.nn as nn
from scipy import signal
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn import preprocessing
from sklearn.metrics import recall_score
from torch.utils.data import TensorDataset, DataLoader

In [2]:
# plot waveform

def plot_waveform(waveform, sample_rate, title="Waveform", xlim=None, ylim=None):
    waveform = waveform.numpy()

    num_channels, num_frames = waveform.shape
    time_axis = torch.arange(0, num_frames) / sample_rate

    figure, axes = plt.subplots(num_channels, 1)
    if num_channels == 1:
        axes = [axes]
    for c in range(num_channels):
        axes[c].plot(time_axis, waveform[c], linewidth=1)
        axes[c].grid(True)
        if num_channels > 1:
            axes[c].set_ylabel(f'Channel {c+1}')
        if xlim:
            axes[c].set_xlim(xlim)
        if ylim:
            axes[c].set_ylim(ylim)
    figure.suptitle(title)
    plt.show(block=False)

In [3]:
def plot_specgram(waveform, sample_rate, title="Spectrogram", xlim=None):
    waveform = waveform.numpy()

    num_channels, num_frames = waveform.shape
    time_axis = torch.arange(0, num_frames) / sample_rate

    figure, axes = plt.subplots(num_channels, 1)
    if num_channels == 1:
        axes = [axes]
    for c in range(num_channels):
        axes[c].specgram(waveform[c], Fs=sample_rate)
        if num_channels > 1:
            axes[c].set_ylabel(f'Channel {c+1}')
        if xlim:
            axes[c].set_xlim(xlim)
    figure.suptitle(title)
    plt.show(block=False)

In [12]:
# data preparation
path = "/Users/liyuanchao/Documents/Corpora/SSPNet/SSPNet/Audio"
os.chdir(path)

train_ex = []
test_ex= []
train_ag = []
test_ag= []
train_co = []
test_co= []
train_ne = []
test_ne= []
train_op = []
test_op= []
feats_train = []
feats_test = []

spectrogram = torchaudio.transforms.Spectrogram()
fmask1 = torchaudio.transforms.FrequencyMasking(freq_mask_param=20)
fmask2 = torchaudio.transforms.FrequencyMasking(freq_mask_param=100)
tmask1 = torchaudio.transforms.TimeMasking(time_mask_param=20)
tmask2 = torchaudio.transforms.TimeMasking(time_mask_param=100)

with open('Scores.csv', 'r') as f:
    file_content = csv.reader(f, delimiter=',')
    headers = next(file_content, None)
    
    for row in list(file_content)[:512]:
        waveform, sr = torchaudio.load(row[0] + '.wav', normalize=True)
        waveform = torch.squeeze(waveform, 0)
        waveform = waveform[1:]
        if len(waveform) < 80000:
            waveform = waveform.numpy()
            waveform = np.pad(waveform, (0, 80000-len(waveform)), 'constant', constant_values=0)
            waveform = torch.from_numpy(waveform)
       
        original = spectrogram(waveform)
        
        fmasked1 = fmask1(original)
        fmasked2 = fmask2(original)
        tmasked1 = tmask1(original)
        tmasked2 = tmask2(original)
                
        feats_train.append(original.numpy())
        train_ex.append(int(row[1]))
        train_ag.append(int(row[2]))
        train_co.append(int(row[3]))
        train_ne.append(int(row[4]))
        train_op.append(int(row[5]))
        
        feats_train.append(fmasked1.numpy())
        train_ex.append(int(row[1]))
        train_ag.append(int(row[2]))
        train_co.append(int(row[3]))
        train_ne.append(int(row[4]))
        train_op.append(int(row[5]))
        
        feats_train.append(fmasked2.numpy())
        train_ex.append(int(row[1]))
        train_ag.append(int(row[2]))
        train_co.append(int(row[3]))
        train_ne.append(int(row[4]))
        train_op.append(int(row[5]))
        
        feats_train.append(tmasked1.numpy())
        train_ex.append(int(row[1]))
        train_ag.append(int(row[2]))
        train_co.append(int(row[3]))
        train_ne.append(int(row[4]))
        train_op.append(int(row[5]))
        
        feats_train.append(tmasked2.numpy())
        train_ex.append(int(row[1]))
        train_ag.append(int(row[2]))
        train_co.append(int(row[3]))
        train_ne.append(int(row[4]))
        train_op.append(int(row[5]))

f.close()
        
with open('Scores.csv', 'r') as f:
    file_content = csv.reader(f, delimiter=',')
    headers = next(file_content, None)
    
    for row in list(file_content)[512:]:
        waveform, sr = torchaudio.load(row[0] + '.wav', normalize=True)
        waveform = torch.squeeze(waveform, 0)
        waveform = waveform[1:]
        if len(waveform) < 80000:
            waveform = waveform.numpy()
            waveform = np.pad(waveform, (0, 80000-len(waveform)), 'constant', constant_values=0)
            waveform = torch.from_numpy(waveform)
       
        original = spectrogram(waveform)
                
        feats_test.append(original.numpy())
        test_ex.append(int(row[1]))
        test_ag.append(int(row[2]))
        test_co.append(int(row[3]))
        test_ne.append(int(row[4]))
        test_op.append(int(row[5]))

f.close()

In [13]:
feats_test = feats_train[int(0.8*3200):]
feats_train = feats_train[:int(0.8*3200)]
test_ex = train_ex[int(0.8*3200):]
test_ag = train_ag[int(0.8*3200):]
test_co = train_co[int(0.8*3200):]
test_ne = train_ne[int(0.8*3200):]
test_op = train_op[int(0.8*3200):]
train_ex = train_ex[:int(0.8*3200)]
train_ag = train_ag[:int(0.8*3200)]
train_co = train_co[:int(0.8*3200)]
train_ne = train_ne[:int(0.8*3200)]
train_op = train_op[:int(0.8*3200)]

In [16]:
feats_train = torch.tensor(feats_train)
feats_test = torch.tensor(feats_test)
train_ex = torch.tensor(train_ex)
train_ag = torch.tensor(train_ag)
train_co = torch.tensor(train_co)
train_ne = torch.tensor(train_ne)
train_op = torch.tensor(train_op)
test_ex = torch.tensor(test_ex)
test_ag = torch.tensor(test_ag)
test_co = torch.tensor(test_co)
test_ne = torch.tensor(test_ne)
test_op = torch.tensor(test_op)

In [17]:
# normalization
feats_train = F.normalize(feats_train, dim=0)
feats_test = F.normalize(feats_test, dim=0)

In [18]:
print(feats_train.size(), feats_test.size())

torch.Size([2560, 201, 401]) torch.Size([640, 201, 401])


In [19]:
trainset = TensorDataset(feats_train, train_ex, train_ag, train_co, train_ne, train_op)
testset = TensorDataset(feats_test, test_ex, test_ag, test_co, test_ne, test_op)

traindata = DataLoader(dataset=trainset, batch_size=64, shuffle=False)
testdata = DataLoader(dataset=testset, batch_size=64, shuffle=False)

print('Data preparation completed!')

Data preparation completed!


In [35]:
# model augmentation
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.cnn = nn.Conv1d(401, 100, kernel_size=3, stride=2)
        self.pool = nn.MaxPool1d(2)
        self.lstm = nn.LSTM(input_size=100,
                            hidden_size=64,
                            num_layers=2,
                            dropout=0.5,
                            batch_first=True,
                            bidirectional=True)
        self.drop = nn.Dropout(p=0.5)
        self.attn = nn.MultiheadAttention(128, 16, batch_first=True)
        self.flat = nn.Flatten()
        self.dense = nn.Linear(10000, 512)
        self.dense2 = nn.Linear(512, 32)
        self.acti = nn.ReLU()
        self.out1 = nn.Linear(32, 2)
        self.out2 = nn.Linear(32, 2)
        self.out3 = nn.Linear(32, 2)
        self.out4 = nn.Linear(32, 2)
        self.out5 = nn.Linear(32, 2)
        
    def forward(self, x):
        # cnn
        x = x.transpose(1,2)
        x = self.cnn(x)
        x = x.transpose(1,2)
#         x = self.pool(x)
        # lstm
#         self.lstm.flatten_parameters()
#         x, _ = self.lstm(x)
#         x = self.drop(x)
        # attention
#         x, _ = self.attn(x,x,x)
        # dense
        x = self.flat(x)
        x = self.drop(x)
        x = self.dense(x)
        x = self.drop(x)
        x = self.dense2(x)
        x = self.drop(x)
        x = self.acti(x)
        # out
#         ex = self.out1(x)
        ag = self.out2(x)
#         co = self.out3(x)
#         ne = self.out4(x)
#         op = self.out5(x)
        return ag

In [38]:
#augmentation

torch.manual_seed(1)
model = NeuralNet()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)
func = nn.CrossEntropyLoss()
# func = nn.BCEWithLogitsLoss(weight=None, size_average=None, reduce=None, reduction='mean', pos_weight=None)

# training
for epoch in range(8):
    start = timeit.default_timer()
    print("-----epoch: ", epoch, "-----")
    loss_list_train_ex = []
    loss_list_train_ag = []
    loss_list_train_co = []
    loss_list_train_ne = []
    loss_list_train_op = []
    loss_list_test_ex = []
    loss_list_test_ag = []
    loss_list_test_co = []
    loss_list_test_ne = []
    loss_list_test_op = []
    
    pred_train_ex = []
    pred_train_ag = []
    pred_train_co = []
    pred_train_ne = []
    pred_train_op = []
    pred_test_ex = []
    pred_test_ag = []
    pred_test_co = []
    pred_test_ne = []
    pred_test_op = []
    
    label_train_ex = []
    label_train_ag = []
    label_train_co = []
    label_train_ne = []
    label_train_op = []
    
    print('--training begins--')
    model.train()
    for feats_input, label_ex, label_ag, label_co, label_ne, label_op in traindata:
        # loss
#         pred_ex, pred_ag, pred_co, pred_ne, pred_op = model(feats_input)
        pred_ag = model(feats_input)
            
#         train_loss_ex = func(pred_ex, label_ex)
        train_loss_ag = func(pred_ag, label_ag)
#         train_loss_co = func(pred_co, label_co)
#         train_loss_ne = func(pred_ne, label_ne)
#         train_loss_op = func(pred_op, label_op)
        
#         loss_list_train_ex.append(train_loss_ex.item())
        loss_list_train_ag.append(train_loss_ag.item())
#         loss_list_train_co.append(train_loss_co.item())
#         loss_list_train_ne.append(train_loss_ne.item())
#         loss_list_train_op.append(train_loss_op.item())
        
#         for i in label_ex:
#             label_train_ex.append(i.detach().numpy())
        for i in label_ag:
            label_train_ag.append(i.detach().numpy())
#         for i in label_co:
#             label_train_co.append(i.detach().numpy())
#         for i in label_ne:
#             label_train_ne.append(i.detach().numpy())
#         for i in label_op:
#             label_train_op.append(i.detach().numpy())

# #         for i in pred_ex:
# #             pred_train_ex.append(i.detach().numpy())
        for i in pred_ag:
            pred_train_ag.append(i.detach().numpy())
#         for i in pred_co:
#             pred_train_co.append(i.detach().numpy())
#         for i in pred_ne:
#             pred_train_ne.append(i.detach().numpy())
#         for i in pred_op:
#             pred_train_op.append(i.detach().numpy())
            
        # backprop
        optimizer.zero_grad()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
#         train_loss = train_loss_ex + train_loss_ag + train_loss_co + train_loss_ne + train_loss_op
        train_loss = train_loss_ag
        train_loss.backward()
        optimizer.step()

# test
    print('--test begins--')
    model.eval()
    for feats_input, label_ex, label_ag, label_co, label_ne, label_op in testdata:
        # loss
#         pred_ex, pred_ag, pred_co, pred_ne, pred_op = model(feats_input)
        pred_ag = model(feats_input)
        
#         test_loss_ex = func(pred_ex, label_ex)
        test_loss_ag = func(pred_ag, label_ag)
#         test_loss_co = func(pred_co, label_co)
#         test_loss_ne = func(pred_ne, label_ne)
#         test_loss_op = func(pred_op, label_op)

#         loss_list_test_ex.append(test_loss_ex.item())
        loss_list_test_ag.append(test_loss_ag.item())
#         loss_list_test_co.append(test_loss_co.item())
#         loss_list_test_ne.append(test_loss_ne.item())
#         loss_list_test_op.append(test_loss_op.item())

#         for i in pred_ex:
#             pred_test_ex.append(i.detach().numpy())
        for i in pred_ag:
            pred_test_ag.append(i.detach().numpy())
#         for i in pred_co:
#             pred_test_co.append(i.detach().numpy())
#         for i in pred_ne:
#             pred_test_ne.append(i.detach().numpy())
#         for i in pred_op:
#             pred_test_op.append(i.detach().numpy())

    # accuracy
#     label_train_ex = np.array(label_train_ex)
    label_train_ag = np.array(label_train_ag)
#     label_train_co = np.array(label_train_co)
#     label_train_ne = np.array(label_train_ne)
#     label_train_op = np.array(label_train_op)
#     pred_train_ex = np.array(pred_train_ex)
    pred_train_ag = np.array(pred_train_ag)
#     pred_train_co = np.array(pred_train_co)
#     pred_train_ne = np.array(pred_train_ne)
#     pred_train_op = np.array(pred_train_op)
#     pred_test_ex = np.array(pred_test_ex)
    pred_test_ag = np.array(pred_test_ag)
#     pred_test_co = np.array(pred_test_co)
#     pred_test_ne = np.array(pred_test_ne)
#     pred_test_op = np.array(pred_test_op)

#     pred_train_ex = [np.argmax(p) for p in pred_train_ex]
    pred_train_ag = [np.argmax(p) for p in pred_train_ag]
#     pred_train_co = [np.argmax(p) for p in pred_train_co]
#     pred_train_ne = [np.argmax(p) for p in pred_train_ne]
#     pred_train_op = [np.argmax(p) for p in pred_train_op]
#     pred_test_ex = [np.argmax(p) for p in pred_test_ex]
    pred_test_ag = [np.argmax(p) for p in pred_test_ag]
#     pred_test_co = [np.argmax(p) for p in pred_test_co]
#     pred_test_ne = [np.argmax(p) for p in pred_test_ne]
#     pred_test_op = [np.argmax(p) for p in pred_test_op]

#     print(np.rint(pred_train_ex))

#     train_acc_ex = recall_score(label_train_ex, pred_train_ex, average='macro')
    train_acc_ag = recall_score(label_train_ag, np.rint(pred_train_ag), average='macro')
#     train_acc_co = recall_score(label_train_co, np.rint(pred_train_co), average='macro')
#     train_acc_ne = recall_score(label_train_ne, np.rint(pred_train_ne), average='macro')
#     train_acc_op = recall_score(label_train_op, np.rint(pred_train_op), average='macro')
#     test_acc_ex = recall_score(test_ex.numpy(), pred_test_ex, average='macro')
    test_acc_ag = recall_score(test_ag.numpy(), np.rint(pred_test_ag), average='macro')
#     test_acc_co = recall_score(test_co.numpy(), np.rint(pred_test_co), average='macro')
#     test_acc_ne = recall_score(test_ne.numpy(), np.rint(pred_test_ne), average='macro')
#     test_acc_op = recall_score(test_op.numpy(), np.rint(pred_test_op), average='macro')

    train_loss = sum(loss_list_train_ag) / len(loss_list_train_ag)


#     (sum(loss_list_train_ex) / len(loss_list_train_ex)) 
#     sum(loss_list_train_ag) / len(loss_list_train_ag)
#     + (sum(loss_list_train_co) / len(loss_list_train_co))
#     + (sum(loss_list_train_ne) / len(loss_list_train_ne))
#     + (sum(loss_list_train_op) / len(loss_list_train_op))
    test_loss = sum(loss_list_test_ag) / len(loss_list_test_ag)


#     (sum(loss_list_test_ex) / len(loss_list_test_ex))
#     (sum(loss_list_test_ag) / len(loss_list_test_ag)
#     + (sum(loss_list_test_co) / len(loss_list_test_co))
#     + (sum(loss_list_test_ne) / len(loss_list_test_ne))
#     + (sum(loss_list_test_op) / len(loss_list_test_op))
        
#     print('train_loss: %.4f' % train_loss, '|test_loss: %.4f' % test_loss, '\n'
#           'train_acc_ex: %.4f' % train_acc_ex, '|train_acc_ag: %.4f' % train_acc_ag, '|train_acc_co: %.4f' % train_acc_co, '|train_acc_ne: %.4f' % train_acc_ne, '|train_acc_op: %.4f' % train_acc_op, '\n'
#           'test_acc_ex: %.4f' % test_acc_ex, '|test_acc_ag: %.4f' % test_acc_ag, '|test_acc_co: %.4f' % test_acc_co, '|test_acc_ne: %.4f' % test_acc_ne, '|test_acc_op: %.4f' % test_acc_op)

    print('train_loss: %.4f' % train_loss, '|test_loss: %.4f' % test_loss, '\n'
          'train_acc_ex: %.4f' % train_acc_ag, 'test_acc_ex: %.4f' % test_acc_ag)

    stop = timeit.default_timer()
    print('Time: ', stop - start)
    scheduler.step()

-----epoch:  0 -----
--training begins--
--test begins--
train_loss: 0.6943 |test_loss: 0.6960 
train_acc_ex: 0.4774 test_acc_ex: 0.5000
Time:  7.308648375000075
-----epoch:  1 -----
--training begins--
--test begins--
train_loss: 0.6910 |test_loss: 0.6957 
train_acc_ex: 0.5000 test_acc_ex: 0.5000
Time:  7.176836500000036
-----epoch:  2 -----
--training begins--
--test begins--
train_loss: 0.6860 |test_loss: 0.6967 
train_acc_ex: 0.5201 test_acc_ex: 0.5177
Time:  7.213723084000094
-----epoch:  3 -----
--training begins--
--test begins--
train_loss: 0.6773 |test_loss: 0.7000 
train_acc_ex: 0.5710 test_acc_ex: 0.5022
Time:  7.237470708000046
-----epoch:  4 -----
--training begins--
--test begins--
train_loss: 0.6570 |test_loss: 0.7011 
train_acc_ex: 0.6250 test_acc_ex: 0.4941
Time:  7.242279000000053
-----epoch:  5 -----
--training begins--
--test begins--
train_loss: 0.6536 |test_loss: 0.7027 
train_acc_ex: 0.6428 test_acc_ex: 0.4818
Time:  7.488836291000098
-----epoch:  6 -----
--train

In [29]:
#augmentation

# torch.manual_seed(1)
# model = NeuralNet()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-6)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)
# func = nn.CrossEntropyLoss()
# func = nn.BCEWithLogitsLoss(weight=None, size_average=None, reduce=None, reduction='mean', pos_weight=None)

# training
for epoch in range(32):
    start = timeit.default_timer()
    print("-----epoch: ", epoch, "-----")
    loss_list_train_ex = []
    loss_list_train_ag = []
    loss_list_train_co = []
    loss_list_train_ne = []
    loss_list_train_op = []
    loss_list_test_ex = []
    loss_list_test_ag = []
    loss_list_test_co = []
    loss_list_test_ne = []
    loss_list_test_op = []
    
    pred_train_ex = []
    pred_train_ag = []
    pred_train_co = []
    pred_train_ne = []
    pred_train_op = []
    pred_test_ex = []
    pred_test_ag = []
    pred_test_co = []
    pred_test_ne = []
    pred_test_op = []
    
    label_train_ex = []
    label_train_ag = []
    label_train_co = []
    label_train_ne = []
    label_train_op = []
    
    print('--training begins--')
    model.train()
    for feats_input, label_ex, label_ag, label_co, label_ne, label_op in traindata:
        # loss
        pred_ex, pred_ag, pred_co, pred_ne, pred_op = model(feats_input)
#         pred_ex = model(feats_input)
            
        train_loss_ex = func(pred_ex, label_ex)
        train_loss_ag = func(pred_ag, label_ag)
        train_loss_co = func(pred_co, label_co)
        train_loss_ne = func(pred_ne, label_ne)
        train_loss_op = func(pred_op, label_op)
        
        loss_list_train_ex.append(train_loss_ex.item())
        loss_list_train_ag.append(train_loss_ag.item())
        loss_list_train_co.append(train_loss_co.item())
        loss_list_train_ne.append(train_loss_ne.item())
        loss_list_train_op.append(train_loss_op.item())
        
        for i in label_ex:
            label_train_ex.append(i.detach().numpy())
        for i in label_ag:
            label_train_ag.append(i.detach().numpy())
        for i in label_co:
            label_train_co.append(i.detach().numpy())
        for i in label_ne:
            label_train_ne.append(i.detach().numpy())
        for i in label_op:
            label_train_op.append(i.detach().numpy())

        for i in pred_ex:
            pred_train_ex.append(i.detach().numpy())
        for i in pred_ag:
            pred_train_ag.append(i.detach().numpy())
        for i in pred_co:
            pred_train_co.append(i.detach().numpy())
        for i in pred_ne:
            pred_train_ne.append(i.detach().numpy())
        for i in pred_op:
            pred_train_op.append(i.detach().numpy())
            
        # backprop
        optimizer.zero_grad()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        train_loss = train_loss_ex + train_loss_ag + train_loss_co + train_loss_ne + train_loss_op
#         train_loss = train_loss_ex
        train_loss.backward()
        optimizer.step()

# test
    print('--test begins--')
    model.eval()
    for feats_input, label_ex, label_ag, label_co, label_ne, label_op in testdata:
        # loss
        pred_ex, pred_ag, pred_co, pred_ne, pred_op = model(feats_input)
#         pred_ex = model(feats_input)
        
        test_loss_ex = func(pred_ex, label_ex)
        test_loss_ag = func(pred_ag, label_ag)
        test_loss_co = func(pred_co, label_co)
        test_loss_ne = func(pred_ne, label_ne)
        test_loss_op = func(pred_op, label_op)

        loss_list_test_ex.append(test_loss_ex.item())
        loss_list_test_ag.append(test_loss_ag.item())
        loss_list_test_co.append(test_loss_co.item())
        loss_list_test_ne.append(test_loss_ne.item())
        loss_list_test_op.append(test_loss_op.item())

        for i in pred_ex:
            pred_test_ex.append(i.detach().numpy())
        for i in pred_ag:
            pred_test_ag.append(i.detach().numpy())
        for i in pred_co:
            pred_test_co.append(i.detach().numpy())
        for i in pred_ne:
            pred_test_ne.append(i.detach().numpy())
        for i in pred_op:
            pred_test_op.append(i.detach().numpy())

    # accuracy
    label_train_ex = np.array(label_train_ex)
    label_train_ag = np.array(label_train_ag)
    label_train_co = np.array(label_train_co)
    label_train_ne = np.array(label_train_ne)
    label_train_op = np.array(label_train_op)
    pred_train_ex = np.array(pred_train_ex)
    pred_train_ag = np.array(pred_train_ag)
    pred_train_co = np.array(pred_train_co)
    pred_train_ne = np.array(pred_train_ne)
    pred_train_op = np.array(pred_train_op)
    pred_test_ex = np.array(pred_test_ex)
    pred_test_ag = np.array(pred_test_ag)
    pred_test_co = np.array(pred_test_co)
    pred_test_ne = np.array(pred_test_ne)
    pred_test_op = np.array(pred_test_op)

    pred_train_ex = [np.argmax(p) for p in pred_train_ex]
    pred_train_ag = [np.argmax(p) for p in pred_train_ag]
    pred_train_co = [np.argmax(p) for p in pred_train_co]
    pred_train_ne = [np.argmax(p) for p in pred_train_ne]
    pred_train_op = [np.argmax(p) for p in pred_train_op]
    pred_test_ex = [np.argmax(p) for p in pred_test_ex]
    pred_test_ag = [np.argmax(p) for p in pred_test_ag]
    pred_test_co = [np.argmax(p) for p in pred_test_co]
    pred_test_ne = [np.argmax(p) for p in pred_test_ne]
    pred_test_op = [np.argmax(p) for p in pred_test_op]

#     print(np.rint(pred_train_ex))

    train_acc_ex = recall_score(label_train_ex, pred_train_ex, average='macro')
    train_acc_ag = recall_score(label_train_ag, np.rint(pred_train_ag), average='macro')
    train_acc_co = recall_score(label_train_co, np.rint(pred_train_co), average='macro')
    train_acc_ne = recall_score(label_train_ne, np.rint(pred_train_ne), average='macro')
    train_acc_op = recall_score(label_train_op, np.rint(pred_train_op), average='macro')
    test_acc_ex = recall_score(test_ex.numpy(), pred_test_ex, average='macro')
    test_acc_ag = recall_score(test_ag.numpy(), np.rint(pred_test_ag), average='macro')
    test_acc_co = recall_score(test_co.numpy(), np.rint(pred_test_co), average='macro')
    test_acc_ne = recall_score(test_ne.numpy(), np.rint(pred_test_ne), average='macro')
    test_acc_op = recall_score(test_op.numpy(), np.rint(pred_test_op), average='macro')

    train_loss = (sum(loss_list_train_ex) / len(loss_list_train_ex)) 
    + (sum(loss_list_train_ag) / len(loss_list_train_ag))
    + (sum(loss_list_train_co) / len(loss_list_train_co))
    + (sum(loss_list_train_ne) / len(loss_list_train_ne))
    + (sum(loss_list_train_op) / len(loss_list_train_op))
    test_loss = (sum(loss_list_test_ex) / len(loss_list_test_ex))
    + (sum(loss_list_test_ag) / len(loss_list_test_ag))
    + (sum(loss_list_test_co) / len(loss_list_test_co))
    + (sum(loss_list_test_ne) / len(loss_list_test_ne))
    + (sum(loss_list_test_op) / len(loss_list_test_op))
        
    print('train_loss: %.4f' % train_loss, '|test_loss: %.4f' % test_loss, '\n'
          'train_acc_ex: %.4f' % train_acc_ex, '|train_acc_ag: %.4f' % train_acc_ag, '|train_acc_co: %.4f' % train_acc_co, '|train_acc_ne: %.4f' % train_acc_ne, '|train_acc_op: %.4f' % train_acc_op, '\n'
          'test_acc_ex: %.4f' % test_acc_ex, '|test_acc_ag: %.4f' % test_acc_ag, '|test_acc_co: %.4f' % test_acc_co, '|test_acc_ne: %.4f' % test_acc_ne, '|test_acc_op: %.4f' % test_acc_op)

#     print('train_loss: %.4f' % train_loss, '|test_loss: %.4f' % test_loss, '\n'
#           'train_acc_ex: %.4f' % train_acc_ex, 'test_acc_ex: %.4f' % test_acc_ex)

    stop = timeit.default_timer()
    print('Time: ', stop - start)
#     scheduler.step()

-----epoch:  0 -----
--training begins--
--test begins--
train_loss: 0.5764 |test_loss: 0.6983 
train_acc_ex: 0.7085 test_acc_ex: 0.5641
Time:  7.2711489579999125
-----epoch:  1 -----
--training begins--
--test begins--
train_loss: 0.5741 |test_loss: 0.6987 
train_acc_ex: 0.7033 test_acc_ex: 0.5641
Time:  7.261592084000085
-----epoch:  2 -----
--training begins--
--test begins--
train_loss: 0.5748 |test_loss: 0.6990 
train_acc_ex: 0.7047 test_acc_ex: 0.5641
Time:  7.292436541999905
-----epoch:  3 -----
--training begins--
--test begins--
train_loss: 0.5735 |test_loss: 0.6993 
train_acc_ex: 0.7049 test_acc_ex: 0.5641
Time:  7.289051208000046
-----epoch:  4 -----
--training begins--
--test begins--
train_loss: 0.5721 |test_loss: 0.6997 
train_acc_ex: 0.7124 test_acc_ex: 0.5641
Time:  7.246960959000035
-----epoch:  5 -----
--training begins--


KeyboardInterrupt: 